In [1]:
import os
import pinecone
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

# openai.api_Key = os.environ.get("OPENAI_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_ENV = os.environ.get("PINECONE_ENVIRONMENT")

/home/charlie/school/code/csci5707/project/venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
index_name = 'song-embeddings-index'
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

index = pinecone.Index(index_name)

In [3]:
index.describe_index_stats()

{'dimension': 11904,
 'index_fullness': 0.03349,
 'namespaces': {'': {'vector_count': 3349}},
 'total_vector_count': 3349}

In [4]:
import numpy as np
import itertools

def chunks(iterable, batch_size=100):
    """Yield successive n-sized chunks from iterable."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

def get_embeddings(directory):
    """Generator to yield (id, vector) pairs from .npy files in the given directory."""
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('_mel_embedding.npy'):
                file_path = os.path.join(root, file)
                embedding = np.load(file_path)
                yield (file.split('_mel_embedding.npy')[0], embedding.tolist())

# Initialize Pinecone
# Use a Pinecone Index with pool_threads for asynchronous requests
with pinecone.Index(index_name, pool_threads=30) as index:
    directories = ["../../data/embeddings/test", "../../data/embeddings/validation", "../../data/embeddings/train"]

    # Process and upsert embeddings asynchronously
    async_results = []
    for directory in directories:
        for batch in chunks(get_embeddings(directory), batch_size=50):
            async_result = index.upsert(vectors=batch, async_req=True)
            async_results.append(async_result)

    # Wait for all asynchronous operations to complete
    for async_result in async_results:
        async_result.get()  # Raises an exception in case of an error

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '133', 'date': 'Sun, 19 Nov 2023 02:34:02 GMT', 'x-envoy-upstream-service-time': '44', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":9,"message":"Starter index record limit reached. Current number: 100076, records in request: 50, limit: 100000","details":[]}


In [ ]:
index.describe_index_stats()